In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hipp

In [ ]:
!rsync -avz ige-calcul1:/mnt/summer/USERS/DEHECQA/history/casa_grande/kh9_pc_images/1978_03_25/raw_images/ /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/

In [16]:
images_directory = "/home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images"
output_directory = "/home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/joined_images"

hipp.kh9pc.join_images(images_directory, output_directory, dryrun=True)


Mosaicking /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/joined_images/D3C1214-100097F013.tif with 10 tiles...

image_mosaic /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_a.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_b.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_c.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_d.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_e.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_f.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-100097F013_g.tif /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9pc/1978_03_25/raw_images/D3C1214-1